In [ ]:
import cv2

def upscale_image_opencv(input_image_path, output_image_path, scale_factor):
    img = cv2.imread(input_image_path)
    new_size = (int(img.shape[1] * scale_factor), int(img.shape[0] * scale_factor))
    upscaled_img = cv2.resize(img, new_size, interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(output_image_path, upscaled_img)


In [ ]:
def calculate_ssim(img, img2, crop_border, input_order='HWC', test_y_channel=False, **kwargs):
    """Calculate SSIM (structural similarity).

    ``Paper: Image quality assessment: From error visibility to structural similarity``

    The results are the same as that of the official released MATLAB code in
    https://ece.uwaterloo.ca/~z70wang/research/ssim/.

    For three-channel images, SSIM is calculated for each channel and then
    averaged.

    Args:
        img (ndarray): Images with range [0, 255].
        img2 (ndarray): Images with range [0, 255].
        crop_border (int): Cropped pixels in each edge of an image. These pixels are not involved in the calculation.
        input_order (str): Whether the input order is 'HWC' or 'CHW'.
            Default: 'HWC'.
        test_y_channel (bool): Test on Y channel of YCbCr. Default: False.

    Returns:
        float: SSIM result.
    """

    assert img.shape == img2.shape, (f'Image shapes are different: {img.shape}, {img2.shape}.')
    if input_order not in ['HWC', 'CHW']:
        raise ValueError(f'Wrong input_order {input_order}. Supported input_orders are "HWC" and "CHW"')
    img = reorder_image(img, input_order=input_order)
    img2 = reorder_image(img2, input_order=input_order)

    if crop_border != 0:
        img = img[crop_border:-crop_border, crop_border:-crop_border, ...]
        img2 = img2[crop_border:-crop_border, crop_border:-crop_border, ...]

    if test_y_channel:
        img = to_y_channel(img)
        img2 = to_y_channel(img2)

    img = img.astype(np.float64)
    img2 = img2.astype(np.float64)

    ssims = []
    for i in range(img.shape[2]):
        ssims.append(_ssim(img[..., i], img2[..., i]))
    return np.array(ssims).mean()

In [ ]:
def calculate_psnr(img, img2, crop_border, input_order='HWC', test_y_channel=False, **kwargs):
    """Calculate PSNR (Peak Signal-to-Noise Ratio).

    Reference: https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio

    Args:
        img (ndarray): Images with range [0, 255].
        img2 (ndarray): Images with range [0, 255].
        crop_border (int): Cropped pixels in each edge of an image. These pixels are not involved in the calculation.
        input_order (str): Whether the input order is 'HWC' or 'CHW'. Default: 'HWC'.
        test_y_channel (bool): Test on Y channel of YCbCr. Default: False.

    Returns:
        float: PSNR result.
    """

    assert img.shape == img2.shape, (f'Image shapes are different: {img.shape}, {img2.shape}.')
    if input_order not in ['HWC', 'CHW']:
        raise ValueError(f'Wrong input_order {input_order}. Supported input_orders are "HWC" and "CHW"')
    img = reorder_image(img, input_order=input_order)
    img2 = reorder_image(img2, input_order=input_order)

    if crop_border != 0:
        img = img[crop_border:-crop_border, crop_border:-crop_border, ...]
        img2 = img2[crop_border:-crop_border, crop_border:-crop_border, ...]

    if test_y_channel:
        img = to_y_channel(img)
        img2 = to_y_channel(img2)

    img = img.astype(np.float64)
    img2 = img2.astype(np.float64)

    mse = np.mean((img - img2)**2)
    if mse == 0:
        return float('inf')
    return 10. * np.log10(255. * 255. / mse)

In [21]:
import os
import cv2
import numpy as np

def upscale_image_opencv(image, scale_factor):
    new_size = (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor))
    upscaled_img = cv2.resize(image, new_size, interpolation=cv2.INTER_NEAREST)
    return upscaled_img

def calculate_psnr(original_img, upscaled_img):
    mse = np.mean((original_img - upscaled_img) ** 2)
    if mse == 0:
        return float('inf')
    psnr = 20 * np.log10(255 / np.sqrt(mse))
    return psnr

def process_images_in_directory(directory_path_lr, directory_path_gt, scale_factor):
    image_files_lr = [f for f in os.listdir(directory_path_lr) if f.endswith(('.png', '.jpg', '.jpeg'))]
    image_files_gt = [f for f in os.listdir(directory_path_gt) if f.endswith(('.png', '.jpg', '.jpeg'))]
    results = {}

    for filename in image_files_lr:
        file_path_lr = os.path.join(directory_path_lr, filename)
        original_img = cv2.imread(file_path_lr)
        
        file_path_gt = os.path.join(directory_path_gt, filename)
        gt_img = cv2.imread(file_path_gt)


        upscaled_img = upscale_image_opencv(original_img, scale_factor)
        psnr_value = calculate_psnr(gt_img, upscaled_img)

        results[filename] = psnr_value
    
    return results

# Use the function like this:
directory_path_lr = '/home/jovyan/work/saved_data/HAT/datasets/Sat3/Sat3Lq'
directory_path_gt = '/home/jovyan/work/saved_data/HAT/datasets/Sat3/Sat3Hq'
scale_factor = 3  # for example, to double the size of the image
results = process_images_in_directory(directory_path_lr, directory_path_gt, scale_factor)
print(results)


{'patch_6.png': 28.75624242862887, 'patch_5.png': 28.36340474090872, 'patch_3.png': 28.781944531765816, 'patch_1.png': 28.706881403837052, 'patch_2.png': 29.15852686990407, 'patch_4.png': 29.042335255187982}


In [18]:
def calculate_mean_psnr(psnr_dict):
    total_sum = sum(psnr_dict.values())
    mean_psnr = total_sum / len(psnr_dict)
    return mean_psnr

calculate_mean_psnr(results)

31.356356539764437

In [ ]:
def calculate_ssim(img, img2):
    ssims = []
    for i in range(img.shape[2]):
        ssims.append(_ssim(img[..., i], img2[..., i]))
    return np.array(ssims).mean()

In [22]:
import os
import cv2
import numpy as np

def upscale_image_opencv(image, scale_factor):
    new_size = (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor))
    upscaled_img = cv2.resize(image, new_size, interpolation=cv2.INTER_NEAREST)
    return upscaled_img

def calculate_ssim(img, img2):
    ssims = []
    for i in range(img.shape[2]):
        ssims.append(_ssim(img[..., i], img2[..., i]))
    return np.array(ssims).mean()

def _ssim(img, img2):
    """Calculate SSIM (structural similarity) for one channel images.

    It is called by func:`calculate_ssim`.

    Args:
        img (ndarray): Images with range [0, 255] with order 'HWC'.
        img2 (ndarray): Images with range [0, 255] with order 'HWC'.

    Returns:
        float: SSIM result.
    """

    c1 = (0.01 * 255)**2
    c2 = (0.03 * 255)**2
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img, -1, window)[5:-5, 5:-5]  # valid mode for window size 11
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + c1) * (2 * sigma12 + c2)) / ((mu1_sq + mu2_sq + c1) * (sigma1_sq + sigma2_sq + c2))
    return ssim_map.mean()


def process_images_in_directory(directory_path_lr, directory_path_gt, scale_factor):
    image_files_lr = [f for f in os.listdir(directory_path_lr) if f.endswith(('.png', '.jpg', '.jpeg'))]
    image_files_gt = [f for f in os.listdir(directory_path_gt) if f.endswith(('.png', '.jpg', '.jpeg'))]
    results = {}

    for filename in image_files_lr:
        file_path_lr = os.path.join(directory_path_lr, filename)
        original_img = cv2.imread(file_path_lr)
        
        file_path_gt = os.path.join(directory_path_gt, filename)
        gt_img = cv2.imread(file_path_gt)


        upscaled_img = upscale_image_opencv(original_img, scale_factor)
        psnr_value = calculate_ssim(gt_img, upscaled_img)

        results[filename] = psnr_value
    
    return results

# Use the function like this:
directory_path_lr = '/home/jovyan/work/saved_data/HAT/datasets/Sat3/Sat3Lq'
directory_path_gt = '/home/jovyan/work/saved_data/HAT/datasets/Sat3/Sat3Hq'
scale_factor = 3  # for example, to double the size of the image
results = process_images_in_directory(directory_path_lr, directory_path_gt, scale_factor)
print(results)

{'patch_6.png': 1.9370440622958427, 'patch_5.png': 1.9728987197560137, 'patch_3.png': 1.955949801890308, 'patch_1.png': 1.8978980633286529, 'patch_2.png': 2.1102403077447214, 'patch_4.png': 1.986388648904062}
